In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import geopandas as gpd

cities = gpd.read_file('500cities_tracts.geojson')
cities.head()

,plctract10,plctrpop10,geometry
0,0107000-01073005910,4612,"(POLYGON ((-86.709181 33.617021, -86.707961 33..."
1,0107000-01073010801,168,"(POLYGON ((-86.74305200000001 33.515411, -86.7..."
2,0107000-01073012701,44,"(POLYGON ((-86.639171 33.525408, -86.639426 33..."
3,0107000-01073012703,498,"(POLYGON ((-86.628861 33.476501, -86.628784 33..."
4,0107000-01073012704,113,"(POLYGON ((-86.630489 33.472661, -86.630545 33..."


In [3]:
# testing with small cities dataset
citiesSample = cities.sort_values(by='plctrpop10', ascending=False)[:30].reset_index(drop=True)
citiesSample

,plctract10,plctrpop10,geometry
0,0613392-06073013310,28960,"(POLYGON ((-116.964576 32.627673, -116.965638 ..."
1,3651000-36005046201,26588,"(POLYGON ((-73.821927 40.870387, -73.821845999..."
2,1245975-12011110325,23101,"(POLYGON ((-80.375777 25.993374, -80.373008 25..."
3,0682590-06071009118,22178,"(POLYGON ((-117.360046 34.506722, -117.360226 ..."
4,1258715-12111382106,21785,"(POLYGON ((-80.37491300000001 27.258435, -80.3..."
5,0636770-06059052420,21098,"(POLYGON ((-117.755544 33.773626, -117.75573 3..."
6,1258715-12111382108,20218,"(POLYGON ((-80.383966 27.374325, -80.383931 27..."
7,4548535-45019004608,19727,"(POLYGON ((-79.790892 32.861828, -79.791213 32..."
8,0682954-06107001003,19471,"(POLYGON ((-119.297016 36.361665, -119.297169 ..."
9,1652120-16001010335,18954,"(POLYGON ((-116.458575 43.64852, -116.454575 4..."


In [4]:
# testing with small set of tweets
tweets = sc.textFile('rachel_tweetsmp.csv')
list(enumerate(tweets.first().split('|')))
# column index 1: latitude
# column index 2: longitude
# column index 5: tweet

[(0, '467753169557000193'),
 (1, '28.18699354'),
 (2, '-82.74002687'),
 (3, 'ItalStal72'),
 (4, 'Sat May 17 19:47:00 +0000 2014'),
 (5,
  "@RickHoffman_ Good show, good actor and the fact you're a @NYRangers fan tops it off."),
 (6, 'actor and fact fan good off show the tops you')]

In [5]:
def cleaningDrugList(txtFile):
    newDrugList = []
    textFile = open(txtFile, "r")
    drugList = textFile.read().split("\n")
    for drug in drugList:
        drugRevised = " {0} ".format(drug)
        newDrugList.append(drugRevised)
    return newDrugList

In [6]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

def cleaningTweet(string):
    import re
    string = deEmojify(string)
    stringLower = string.lower()
    cleanedString = re.sub('[^0-9a-zA-Z]+', ' ', stringLower)
    cleanedStringLength = len(cleanedString)+2
    cleanedStringRevised = cleanedString.center(cleanedStringLength)
    return cleanedStringRevised

In [7]:
def createIndex(geojson):
    import rtree
    import fiona.crs
    import geopandas as gpd
    citiesSample = gpd.read_file(geojson).sort_values(by='plctrpop10', ascending=False)[:30].reset_index(drop=True)
    index = rtree.Rtree()
    for idx,geometry in enumerate(citiesSample.geometry):
        index.insert(idx, geometry.bounds)
    return(index, citiesSample)

In [8]:
def findTract(p, index, tracts):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if tracts.geometry[idx].contains(p):
            return idx

In [9]:
def tweetFilter(pid, records):
    illegal = cleaningDrugList("drug_illegal.txt")
    sched = cleaningDrugList("drug_sched2.txt")
    drugsList = illegal+sched
    indexT, tractsT = createIndex('500cities_tracts.geojson')
    tractName = tractsT['plctract10']
    tractPop = tractsT['plctrpop10']
    import re
    for line in records:
        row = line.split('|')
        if len(row) != 7:
            continue
        else:
            cleanedTweet = cleaningTweet(row[5])
            words_re = re.compile("|".join(drugsList))
            if words_re.search(cleanedTweet):
                import shapely.geometry as geom
                try:
                    lon = float(row[2])
                except:
                    continue
                try:
                    lat = float(row[1])
                except:
                    continue
                tract = None
                try:
                    loc = geom.Point((float(lon), float(lat)))
                except:
                    continue
                try:
                    tract = findTract(loc, indexT, tractsT)
                except:
                    continue
                if tract is not None:
                    if int(tractPop[tract])>0:
                        yield ((tractName[tract], tractPop[tract]), 1)

In [10]:
test = tweets.mapPartitionsWithIndex(tweetFilter) \
                .filter(lambda x: x is not None) \
                .reduceByKey(lambda a,b: a+b) \
                .map(lambda x: (x[0][0], x[1]/x[0][1])) \
                .sortByKey(ascending=True) \
                .collect()

print(test)

[('0636770-06059052420', 4.7397857616835716e-05), ('0675000-06077003500', 0.00013700506918755994), ('0682590-06071009110', 6.75949709341625e-05), ('0682954-06107001003', 5.1358430486364334e-05), ('1258715-12111382111', 7.027900766041184e-05), ('1714000-17031330100', 0.00011951000896325067), ('3651000-36061024500', 6.696129637069774e-05)]
